# Test ETradeBot

In [1]:
import ast
import sys
import keyring

In [2]:
sys.path.insert(0, '../')
from strategies.strategy import Strategy
from etrade.etrade import ETrade
from execute.execute import Execute
from bot import Bot

## Authentication

In [3]:
# Retrieve credentials from keyring
consumer_key = keyring.get_password("etrade", "consumer_key")
consumer_secret = keyring.get_password("etrade", "consumer_secret")
sandbox_key = keyring.get_password("etrade", "sandbox_key")
sandbox_secret = keyring.get_password("etrade", "sandbox_secret")
web_username = keyring.get_password("etrade", "web_username")
web_password = keyring.get_password("etrade", "web_password")
account_id = keyring.get_password("etrade", "account_id")
etrade_cookie = ast.literal_eval(keyring.get_password("etrade", "cookie"))
account_id_key = keyring.get_password("etrade", "account_id_key")

In [5]:
# Define the strategy name
strategy_name = 'cape_strategy'

In [6]:
# Configuration
dev=False  # Set to False for production, True for sandbox
headless=True  # Set to False to show browser window, True to hide
browser='chrome'  # Set to 'chrome' or 'edge'
preview=True  # Set to False to execute trades, True to preview
prints=True  # Set to True to print information to console

In [7]:
# Initialize API objects
etrade = ETrade(
    consumer_key, consumer_secret, web_username, web_password, account_id, etrade_cookie,
    sandbox_key, sandbox_secret, dev, headless, browser
)
strategy = Strategy(prints, strategy_name)
execute = Execute(etrade)

## Strategy

In [8]:
new_portfolio = strategy.run_strategy()

------------------------------------
Risk Free Rate: 3.34%

Market Cap ($Millions):
IJR     $65,270.56
IVE     $24,296.00
SPYV    $14,531.98
IUSV    $13,173.88
IJS      $6,831.28
SPSM     $4,953.58
IJT      $4,903.15
SLYV     $3,809.86
VOOV     $3,172.67
OIH      $2,400.88
SLYG     $2,324.20
VIOO     $2,218.23
SLY      $1,695.18
VIOV     $1,265.76
FYX        $785.47
VIOG       $506.40
PSCD        $27.79
Name: CUR_MKT_CAP, dtype: object

VIF Tickers:
TICKER
OIH      2.36
PSCD     4.39
SPYV     5.00
VIOV    23.44
VIOG    23.69
SPSM    29.99
FYX     32.32
Name: VIF Factor, dtype: float64

Investor Views:
TICKER
VIOV     23.39%
PSCD     23.03%
FYX      21.78%
SPSM     21.78%
VIOG     19.57%
SPYV     -0.85%
OIH     -11.73%
Name: Investor Views, dtype: object

View Confidences:
TICKER
VIOV    73.60%
FYX     64.98%
SPSM    64.98%
VIOG    62.82%
PSCD    57.29%
SPYV    48.32%
OIH     32.48%
Name: Confidences, dtype: object
Market Symbol: SPY US Equity
Market Name: SPDR S&P 500 ETF Trust

Market

In [9]:
new_portfolio

VIOV    0.5449
VIOG    0.3281
PSCD    0.1743
SPSM    0.1666
FYX     0.0861
IJT    -0.1496
OIH    -0.1504
Name: Maximum Sharpe Portfolio, dtype: float64

## Current Portfolio

In [10]:
# Get current portfolio
current_portfolio = etrade.get_portfolio_data(account_id_key)[[
    'pctOfPortfolio', 'quantity', 'positionType'
]]
current_portfolio

,pctOfPortfolio,quantity,positionType
symbolDescription,,,
OIH,-0.143620,-5,SHORT
IJT,-0.152696,-14,SHORT
VIOV,0.543875,68,LONG
PSCD,0.175593,20,LONG
VIOG,0.328792,35,LONG
FYX,0.088235,11,LONG
SPSM,0.164816,44,LONG


In [11]:
# Get buying power
buying_power = etrade.get_buying_power(account_id_key, prints=prints)


Buying Power: $9,891.01


## Implement Strategy

In [12]:
# Calculate new portfolio shares
new_portfolio_shares = execute.calculate_shares(new_portfolio, buying_power, prints=prints)


Trade Instructions:
VIOV     $5,379.48
VIOG     $3,159.17
PSCD     $1,736.80
SPSM     $1,667.25
FYX        $872.74
OIH     $-1,420.55
IJT     $-1,510.32
Name: Trade Amounts, dtype: object

Position Value: $9,884.57
Leftover: $79.30

VIOV     68
VIOG     34
PSCD     20
SPSM     45
FYX      11
OIH      -5
IJT     -14
Name: Shares, dtype: object


In [13]:
# Preview or execute trades
print('')
if preview:
    print('Previewing trades...')
else:
    print('Executing trades...')
place_order = execute.execute_trades(
    current_portfolio, new_portfolio_shares, account_id_key, preview, prints
)


Previewing trades...
SELL -1 shares of VIOG
BUY 1 shares of SPSM


In [14]:
place_order

,orderAction,priceType,quantity,orderTerm,marketSession
symbol,,,,,
VIOG,SELL,MARKET,1,GOOD_FOR_DAY,REGULAR
SPSM,BUY,MARKET,1,GOOD_FOR_DAY,REGULAR
